In [1]:
import xarray as xr
import pandas as pd
from monetio.models import cmaq
from scipy import interpolate
import geopandas

# using geocube_env python environment (python 3.11)


Please install h5py to open files from the Amazon S3 servers.
Please install h5netcdf to open files from the Amazon S3 servers.


In [2]:
def interpolate_cmaq_census(cmaq_file: str, census_file: str) -> pd.DataFrame:
    """Perform cubic spline interpolation of CMAQ PM2.5 output to census tract centers of population.

    Args:
        cmaq_file (str): Path to CMAQ pseudonetcdf file.
        census_file (str): Path to census data point locations.

    Returns:
        pd.DataFrame: Census location input with PM2.5 field added.
    """
    ds = cmaq.open_dataset(fname=cmaq_file)

    # get annual average PM2.5
    dapm = (
        ds["PM25_AVG"].mean(dim="time").mean(dim="z")
    )  # taking mean of 1 level z to drop it

    census_points = geopandas.read_file(census_file)

    # transform pm2.5 and lat/long data into tidy dataframe
    vals = [
        dapm.values,
        dapm.coords["longitude"].values,
        dapm.coords["latitude"].values,
    ]

    pm25_df = pd.DataFrame(
        [pd.DataFrame(x).stack() for x in vals],
        index=["PM2.5", "longitude", "latitude"],
    ).T

    # perform interpolation of annual average pm2.5 data to census points
    census_points["pm25"] = interpolate.griddata(
        points=pm25_df[["longitude", "latitude"]],
        values=pm25_df["PM2.5"],
        xi=census_points[["LONGITUDE", "LATITUDE"]],
        method="cubic",
    )

    return census_points


In [3]:
cmaq_file = "C:\\Users\\rrice\\OneDrive - Environmental Protection Agency (EPA)\\exposure disparities\\EQUATES data\\HR2DAY_LST_ACONC_EQUATES_v532_12US1_2010.nc"
census_file = "C:\\Users\\rrice\\OneDrive - Environmental Protection Agency (EPA)\\exposure disparities\\nhgis0002_shape\\nhgis0002_shapefile_cenpop2010_us_blck_grp_cenpop_2010\\US_blck_grp_cenpop_2010.shp"

census_points = interpolate_cmaq_census(cmaq_file, census_file)


c:\Users\rrice\Miniconda3\envs\geocube_env\Lib\site-packages\pyproj\crs\crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)


<xarray.Dataset>
Dimensions:       (TSTEP: 365, VAR: 14, DATE-TIME: 2, LAY: 1, ROW: 299, COL: 459)
Dimensions without coordinates: TSTEP, VAR, DATE-TIME, LAY, ROW, COL
Data variables: (12/15)
    TFLAG         (TSTEP, VAR, DATE-TIME) int32 ...
    O3_MDA8       (TSTEP, LAY, ROW, COL) float32 ...
    O3_AVG        (TSTEP, LAY, ROW, COL) float32 ...
    CO_AVG        (TSTEP, LAY, ROW, COL) float32 ...
    NO_AVG        (TSTEP, LAY, ROW, COL) float32 ...
    NO2_AVG       (TSTEP, LAY, ROW, COL) float32 ...
    ...            ...
    PM25_AVG      (TSTEP, LAY, ROW, COL) float32 ...
    PM25_SO4_AVG  (TSTEP, LAY, ROW, COL) float32 ...
    PM25_NO3_AVG  (TSTEP, LAY, ROW, COL) float32 ...
    PM25_NH4_AVG  (TSTEP, LAY, ROW, COL) float32 ...
    PM25_OC_AVG   (TSTEP, LAY, ROW, COL) float32 ...
    PM25_EC_AVG   (TSTEP, LAY, ROW, COL) float32 ...
Attributes: (12/34)
    IOAPI_VERSION:  $Id: @(#) ioapi library version 3.1 $                    ...
    EXEC_ID:        ????????????????             

In [4]:
census_points


,GISJOIN,GEOID,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,POPULATION,LATITUDE,LONGITUDE,geometry,pm25
0,G01000100201001,010010201001,01,001,020100,1,698,32.464812,-86.486527,POINT (887998.982 -518710.748),11.342387
1,G01000100201002,010010201002,01,001,020100,2,1214,32.482391,-86.486912,POINT (887767.270 -516763.948),11.277763
2,G01000100202001,010010202001,01,001,020200,1,1003,32.478035,-86.474786,POINT (888943.691 -517133.898),11.416936
3,G01000100202002,010010202002,01,001,020200,2,1167,32.466372,-86.471060,POINT (889420.440 -518393.029),11.503473
4,G01000100203001,010010203001,01,001,020300,1,2549,32.476828,-86.460326,POINT (890302.131 -517132.451),11.597906
...,...,...,...,...,...,...,...,...,...,...,...
220329,G72015307506011,721537506011,72,153,750601,1,1335,18.016058,-66.833965,POINT (3161611.954 -1654006.890),NaN
220330,G72015307506012,721537506012,72,153,750601,2,2986,18.017193,-66.847551,POINT (3160149.245 -1654343.611),NaN
220331,G72015307506013,721537506013,72,153,750601,3,994,18.023187,-66.848003,POINT (3159909.147 -1653750.449),NaN
220332,G72015307506021,721537506021,72,153,750602,1,1872,18.012677,-66.869466,POINT (3157993.583 -1655530.334),NaN


In [5]:
import plotly.express as px

# px.scatter_geo(
#    data_frame=census_points, lon="LONGITUDE", lat="LATITUDE", color="pm25", scope="usa",
# )


In [6]:
adi_path = "C:\\Users\\rrice\\OneDrive - Environmental Protection Agency (EPA)\\exposure disparities\\adi-download\\US_2020_ADI_Census Block Group_v3.2.csv"
adi_df = pd.read_csv(adi_path)
adi_df

,OBJECTID,GISJOIN,ADI_NATRANK,ADI_STATERNK,FIPS
0,1,G01000100201001,73,5,10010201001
1,2,G01000100201002,62,3,10010201002
2,3,G01000100202001,83,7,10010202001
3,4,G01000100202002,87,7,10010202002
4,5,G01000100203001,73,5,10010203001
...,...,...,...,...,...
242330,242331,G72015307506011,92,6,721537506011
242331,242332,G72015307506012,87,4,721537506012
242332,242333,G72015307506013,93,7,721537506013
242333,242334,G72015307506021,98,10,721537506021


In [10]:
# TODO: join census demographic information

dem_df = pd.read_csv(
    "C:\\Users\\rrice\\OneDrive - Environmental Protection Agency (EPA)\\exposure disparities\\nhgis0002_csv\\nhgis0002_ds172_2010_blck_grp.csv",
    encoding="cp1252",
).drop(
    0
)  # read census data, drop first line of data descriptions
dem_df


C:\Users\rrice\AppData\Local\Temp\ipykernel_15736\439290152.py:3: DtypeWarning: Columns (1,3,4,6,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,51,52,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89) have mixed types. Specify dtype option on import or set low_memory=False.
  dem_df = pd.read_csv("C:\\Users\\rrice\\OneDrive - Environmental Protection Agency (EPA)\\exposure disparities\\nhgis0002_csv\\nhgis0002_ds172_2010_blck_grp.csv", encoding = 'cp1252').drop(0)


,GISJOIN,YEAR,STUSAB,REGIONA,DIVISIONA,STATE,STATEA,COUNTY,COUNTYA,COUSUBA,...,H7Z008,H7Z009,H7Z010,H7Z011,H7Z012,H7Z013,H7Z014,H7Z015,H7Z016,H7Z017
1,G01000100201001,2010,AL,3,6,Alabama,01,Autauga County,001,NaN,...,0,10,18,5,0,1,0,0,6,6
2,G01000100201002,2010,AL,3,6,Alabama,01,Autauga County,001,NaN,...,1,12,26,16,0,0,0,0,7,3
3,G01000100202001,2010,AL,3,6,Alabama,01,Autauga County,001,NaN,...,1,19,31,10,1,0,0,0,20,0
4,G01000100202002,2010,AL,3,6,Alabama,01,Autauga County,001,NaN,...,2,5,44,34,2,0,0,0,4,4
5,G01000100203001,2010,AL,3,6,Alabama,01,Autauga County,001,NaN,...,3,56,69,35,0,2,0,0,19,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220330,G72015307506011,2010,PR,9,0,Puerto Rico,72,Yauco Municipio,153,NaN,...,0,0,1328,864,106,3,4,0,283,68
220331,G72015307506012,2010,PR,9,0,Puerto Rico,72,Yauco Municipio,153,NaN,...,0,0,2965,2453,201,5,1,0,201,104
220332,G72015307506013,2010,PR,9,0,Puerto Rico,72,Yauco Municipio,153,NaN,...,0,0,991,859,39,1,1,0,55,36
220333,G72015307506021,2010,PR,9,0,Puerto Rico,72,Yauco Municipio,153,NaN,...,0,0,1865,1538,159,16,0,0,116,36


In [30]:
joined = (
    census_points.set_index(["GISJOIN"])
    .join(dem_df.set_index(["GISJOIN"]), how="outer") # this join is good - same number of rows in each census_points and dem_df
    .join(adi_df.set_index(["GISJOIN"]), how="left") #TODO: figure out why there are more adi census block groups than from the census files?
    .drop("geometry", axis="columns")
)
joined


,GEOID,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,POPULATION,LATITUDE,LONGITUDE,pm25,YEAR,...,H7Z012,H7Z013,H7Z014,H7Z015,H7Z016,H7Z017,OBJECTID,ADI_NATRANK,ADI_STATERNK,FIPS
GISJOIN,,,,,,,,,,,,,,,,,,,,,
G01000100201001,010010201001,01,001,020100,1,698,32.464812,-86.486527,11.342387,2010,...,0,1,0,0,6,6,1.0,73,5,1.001020e+10
G01000100201002,010010201002,01,001,020100,2,1214,32.482391,-86.486912,11.277763,2010,...,0,0,0,0,7,3,2.0,62,3,1.001020e+10
G01000100202001,010010202001,01,001,020200,1,1003,32.478035,-86.474786,11.416936,2010,...,1,0,0,0,20,0,3.0,83,7,1.001020e+10
G01000100202002,010010202002,01,001,020200,2,1167,32.466372,-86.471060,11.503473,2010,...,2,0,0,0,4,4,4.0,87,7,1.001020e+10
G01000100203001,010010203001,01,001,020300,1,2549,32.476828,-86.460326,11.597906,2010,...,0,2,0,0,19,13,5.0,73,5,1.001020e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
G72015307506011,721537506011,72,153,750601,1,1335,18.016058,-66.833965,NaN,2010,...,106,3,4,0,283,68,242331.0,92,6,7.215375e+11
G72015307506012,721537506012,72,153,750601,2,2986,18.017193,-66.847551,NaN,2010,...,201,5,1,0,201,104,242332.0,87,4,7.215375e+11
G72015307506013,721537506013,72,153,750601,3,994,18.023187,-66.848003,NaN,2010,...,39,1,1,0,55,36,242333.0,93,7,7.215375e+11


In [31]:
adi_df["GISJOIN"]

0         G01000100201001
1         G01000100201002
2         G01000100202001
3         G01000100202002
4         G01000100203001
               ...       
242330    G72015307506011
242331    G72015307506012
242332    G72015307506013
242333    G72015307506021
242334    G72015307506022
Name: GISJOIN, Length: 242335, dtype: object

In [32]:
census_points["GISJOIN"]

0         G01000100201001
1         G01000100201002
2         G01000100202001
3         G01000100202002
4         G01000100203001
               ...       
220329    G72015307506011
220330    G72015307506012
220331    G72015307506013
220332    G72015307506021
220333    G72015307506022
Name: GISJOIN, Length: 220334, dtype: object

In [33]:
dem_df["GISJOIN"]

1         G01000100201001
2         G01000100201002
3         G01000100202001
4         G01000100202002
5         G01000100203001
               ...       
220330    G72015307506011
220331    G72015307506012
220332    G72015307506013
220333    G72015307506021
220334    G72015307506022
Name: GISJOIN, Length: 220334, dtype: object

In [34]:
joined.to_csv("data/biol562 project dataset v1.csv")